In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing 
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Train.csv to Train.csv


In [ ]:
df = pd.read_csv('Train.csv')
df

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,Male,No,22,No,NaN,0.0,Low,7.0,Cat_1,D
8064,464685,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4,D
8065,465406,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6,D
8066,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,B


In [ ]:
df.isna().sum()

ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64

Размер семьи может зависить от фактора брака. Если люди в браке, то семья больше так как в браке будут дети.

In [ ]:
mean_ever_married_true = df.groupby('Ever_Married').mean()['Family_Size'].round().loc['Yes']
mean_ever_married_false = df.groupby('Ever_Married').mean()['Family_Size'].round().loc['No']
print (mean_ever_married_true, mean_ever_married_false)

3.0 3.0


Видим, что значения одинаковые, следовательно нет смысла групировать по фактору брака. Тогда запишим вместо пустых значений просто среднее значение (3.0).

In [ ]:
df['Family_Size'] = df['Family_Size'].fillna(3.0)

Посмотрим сколько каких значений в столбце Segmentation

In [ ]:
df['Segmentation'].value_counts()

D    2268
A    1972
C    1970
B    1858
Name: Segmentation, dtype: int64

Заменим категориальные признаки в столбце Segmentation

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['Segmentation'])
df['Segmentation'] = label_encoder.transform(df['Segmentation']) 

In [ ]:
df

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,3
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,0
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,1
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,1
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,0
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,Male,No,22,No,NaN,0.0,Low,7.0,Cat_1,3
8064,464685,Male,No,35,No,Executive,3.0,Low,4.0,Cat_4,3
8065,465406,Female,No,33,Yes,Healthcare,1.0,Low,1.0,Cat_6,3
8066,467299,Female,No,27,Yes,Healthcare,1.0,Low,4.0,Cat_6,1


In [ ]:
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['Segmentation'])
df['Segmentation'] = label_encoder.transform(df['Segmentation']) 

Заменим ещё в нескольких столбцах категориальные признаки

In [ ]:
df['Spending_Score'].value_counts()

Low        4878
Average    1974
High       1216
Name: Spending_Score, dtype: int64

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['Spending_Score'])
df['Spending_Score'] = label_encoder.transform(df['Spending_Score']) 
df

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,2,4.0,Cat_4,3
1,462643,Female,Yes,38,Yes,Engineer,NaN,0,3.0,Cat_4,0
2,466315,Female,Yes,67,Yes,Engineer,1.0,2,1.0,Cat_6,1
3,461735,Male,Yes,67,Yes,Lawyer,0.0,1,2.0,Cat_6,1
4,462669,Female,Yes,40,Yes,Entertainment,NaN,1,6.0,Cat_6,0
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,Male,No,22,No,NaN,0.0,2,7.0,Cat_1,3
8064,464685,Male,No,35,No,Executive,3.0,2,4.0,Cat_4,3
8065,465406,Female,No,33,Yes,Healthcare,1.0,2,1.0,Cat_6,3
8066,467299,Female,No,27,Yes,Healthcare,1.0,2,4.0,Cat_6,1


Строки с пустыми значениями в столбцах Graduated и Var_1 можно просто удалить так как там малое количество таких строк

In [ ]:
df = df.drop(df.loc[df['Graduated'].isna()].index, axis=0)
df = df.drop(df.loc[df['Var_1'].isna()].index, axis=0)

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['Graduated'])
df['Graduated'] = label_encoder.transform(df['Graduated']) 

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['Gender'])
df['Gender'] = label_encoder.transform(df['Gender']) 

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['Var_1'])
df['Var_1'] = label_encoder.transform(df['Var_1'])

In [ ]:
df

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,1,No,22,0,Healthcare,1.0,2,4.0,3,3
1,462643,0,Yes,38,1,Engineer,NaN,0,3.0,3,0
2,466315,0,Yes,67,1,Engineer,1.0,2,1.0,5,1
3,461735,1,Yes,67,1,Lawyer,0.0,1,2.0,5,1
4,462669,0,Yes,40,1,Entertainment,NaN,1,6.0,5,0
...,...,...,...,...,...,...,...,...,...,...,...
8063,464018,1,No,22,0,NaN,0.0,2,7.0,0,3
8064,464685,1,No,35,0,Executive,3.0,2,4.0,3,3
8065,465406,0,No,33,1,Healthcare,1.0,2,1.0,5,3
8066,467299,0,No,27,1,Healthcare,1.0,2,4.0,5,1


Пустые значения в столбце   заполним просто среднем значением по столбцу  (предварительно проверяли зависимости от других столбцов. Они оказались очень слабыми)

In [ ]:
mean_value = df['Work_Experience'].mean()
df['Work_Experience'] = df['Work_Experience'].fillna(mean_value)

Также пустыми остались пустые значения в строках Ever_Married (135 путых значений) и Profession (114 путых значений). При этом датасете имеем 8067 строк, тогда если мы удалим строки с пустыми значениями, мы потеряем информацию примерно 0,01% людях. Такой информацией можно пренебречь (маленький процент). Удалим в этих столбцах пустые значения.

In [ ]:
df = df.drop(df.loc[df['Ever_Married'].isna()].index, axis=0)
df = df.drop(df.loc[df['Profession'].isna()].index, axis=0)

Заново проверим данные на пустые значения

In [ ]:
df.isna().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

In [ ]:
df

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,1,No,22,0,Healthcare,1.000000,2,4.0,3,3
1,462643,0,Yes,38,1,Engineer,2.636504,0,3.0,3,0
2,466315,0,Yes,67,1,Engineer,1.000000,2,1.0,5,1
3,461735,1,Yes,67,1,Lawyer,0.000000,1,2.0,5,1
4,462669,0,Yes,40,1,Entertainment,2.636504,1,6.0,5,0
...,...,...,...,...,...,...,...,...,...,...,...
8062,463002,1,Yes,41,1,Artist,0.000000,1,5.0,5,1
8064,464685,1,No,35,0,Executive,3.000000,2,4.0,3,3
8065,465406,0,No,33,1,Healthcare,1.000000,2,1.0,5,3
8066,467299,0,No,27,1,Healthcare,1.000000,2,4.0,5,1


Отлично! Мы полностью избавились от пустых значений. Преобразуем оставшиеся столбцы с категориальными признаками.

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['Ever_Married'])
df['Ever_Married'] = label_encoder.transform(df['Ever_Married']) 

In [ ]:
label_encoder = preprocessing.LabelEncoder() 
label_encoder.fit(df['Profession'])
df['Profession'] = label_encoder.transform(df['Profession']) 

Выведем информацию о получившемся датасете

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7669 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               7669 non-null   int64  
 1   Gender           7669 non-null   int64  
 2   Ever_Married     7669 non-null   int64  
 3   Age              7669 non-null   int64  
 4   Graduated        7669 non-null   int64  
 5   Profession       7669 non-null   int64  
 6   Work_Experience  7669 non-null   float64
 7   Spending_Score   7669 non-null   int64  
 8   Family_Size      7669 non-null   float64
 9   Var_1            7669 non-null   int64  
 10  Segmentation     7669 non-null   int64  
dtypes: float64(2), int64(9)
memory usage: 719.0 KB


Отлично! В данном датасете полностью отсутствуют как нулевые значения, так и категориальные признаки.

Продолжим подготовку данных. Теперь можно, например, провести масштабирование признаков. В данном датасете масштабирования требует только колонка Age (Остальные значения в столюцах имеют деапазон примерно 0-10).

In [ ]:
df

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,1,0,22,0,5,1.000000,2,4.0,3,3
1,462643,0,1,38,1,2,2.636504,0,3.0,3,0
2,466315,0,1,67,1,2,1.000000,2,1.0,5,1
3,461735,1,1,67,1,7,0.000000,1,2.0,5,1
4,462669,0,1,40,1,3,2.636504,1,6.0,5,0
...,...,...,...,...,...,...,...,...,...,...,...
8062,463002,1,1,41,1,0,0.000000,1,5.0,5,1
8064,464685,1,0,35,0,4,3.000000,2,4.0,3,3
8065,465406,0,0,33,1,5,1.000000,2,1.0,5,3
8066,467299,0,0,27,1,5,1.000000,2,4.0,5,1


In [ ]:
X =  df.drop(['Ever_Married'], axis = 1)
y = df['Ever_Married']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X)
X_norm

array([[0.42560053, 1.        , 0.05633803, ..., 0.375     , 0.5       ,
        1.        ],
       [0.40713968, 0.        , 0.28169014, ..., 0.25      , 0.5       ,
        0.        ],
       [0.81550267, 0.        , 0.69014085, ..., 0.        , 0.83333333,
        0.33333333],
       ...,
       [0.71441281, 0.        , 0.21126761, ..., 0.        , 0.83333333,
        1.        ],
       [0.92493327, 0.        , 0.12676056, ..., 0.375     , 0.83333333,
        0.33333333],
       [0.32217527, 1.        , 0.26760563, ..., 0.25      , 0.5       ,
        0.33333333]])

#дерево решений для нормализованных данных 

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_norm, y, test_size=1/3, random_state=42)

tree = DecisionTreeClassifier()


tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)


print("Accuracy" , accuracy_score(y_test, y_pred_tree))
print("Recall" ,recall_score(y_test, y_pred_tree))
print("Precision" , precision_score(y_test, y_pred_tree)) 

Accuracy 0.8517794290183809
Recall 0.871523178807947
Precision 0.8767488341105929


#дерево решений с помощью кросс-валидации(норм. данные)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_norm, y, test_size=1/3, random_state=42)

tree = DecisionTreeClassifier()


tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)


cvs = cross_val_score(estimator = model, X = X_norm, y = y, cv = kf, scoring = 'accuracy')

print(cvs)
print(cvs.mean())

Accuracy 0.8517794290183809
Recall 0.8721854304635761
Precision 0.8762475049900199


#дерево решений для стандартизированных данных 

In [ ]:
scaler = StandardScaler()
X_st = scaler.fit_transform(X)
X_st

array([[-0.25661788,  0.90476042, -1.28884403, ...,  0.76518376,
        -0.817195  ,  1.27008395],
       [-0.32069403, -1.10526497, -0.33008515, ...,  0.0979644 ,
        -0.817195  , -1.37313826],
       [ 1.09670132, -1.10526497,  1.40766531, ..., -1.23647431,
         0.59414797, -0.49206419],
       ...,
       [ 0.74582649, -1.10526497, -0.6296973 , ..., -1.23647431,
         0.59414797,  1.27008395],
       [ 1.47652622, -1.10526497, -0.98923188, ...,  0.76518376,
         0.59414797, -0.49206419],
       [-0.61559872,  0.90476042, -0.39000758, ...,  0.0979644 ,
        -0.817195  , -0.49206419]])

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_st, y, test_size=1/3, random_state=42)

tree = DecisionTreeClassifier()


tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)


print("Accuracy" , accuracy_score(y_test, y_pred_tree))
print("Recall" ,recall_score(y_test, y_pred_tree))
print("Precision" , precision_score(y_test, y_pred_tree)) 

Accuracy 0.8572545952287838
Recall 0.8741721854304636
Precision 0.882943143812709


#дерево решений с помощью кросс-валидации(ст. данные)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_st, y, test_size=1/3, random_state=42)

tree = DecisionTreeClassifier()


tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)

cvs = cross_val_score(estimator = model, X = X_norm, y = y, cv = kf, scoring = 'accuracy')

print(cvs)
print(cvs.mean())

Accuracy 0.8533437622213531
Recall 0.8748344370860928
Precision 0.8765759787657598


#логистическая регрессия для нормализованных данных

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_norm,y, test_size=0.3, random_state = 42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy" , accuracy_score(y_test, y_pred))
print("Recall" ,recall_score(y_test, y_pred))
print("Precision" , precision_score(y_test, y_pred)) 

Accuracy 0.8900478053020426
Recall 0.8585043988269795
Precision 0.9512591389114541


#логистическая регрессия для стандартизированных данных

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_st,y, test_size=0.3, random_state = 42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy" , accuracy_score(y_test, y_pred))
print("Recall" ,recall_score(y_test, y_pred))
print("Precision" , precision_score(y_test, y_pred)) 

Accuracy 0.8904823989569752
Recall 0.8599706744868035
Precision 0.9505672609400324


# метод ближайших соседей (для стандартизированных  данных)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_st, y, test_size=0.3, random_state = 42)

scores = []
for i in range(1,15):

  knn = KNeighborsClassifier(i)
  knn.fit(X_train, y_train)
  answers = knn.predict(X_test)

  scores.append(knn.score(X_test, y_test))

print(scores)


max_score = max(scores)
scores_ind = [i for i, v in enumerate(scores) if v == max_score]
scores_ind


knn = KNeighborsClassifier(3)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

confusion_matrix(y_test, y_pred)

print("Accuracy" , accuracy_score(y_test, y_pred))
print("Recall" ,recall_score(y_test, y_pred))
print("Precision" , precision_score(y_test, y_pred)) 


[0.8383311603650587, 0.8426770969143851, 0.8600608431116906, 0.8504997827031725, 0.8639721860060843, 0.85832246849196, 0.8670143415906127, 0.8626684050412864, 0.8622338113863538, 0.8565840938722294, 0.8600608431116906, 0.8609300304215558, 0.8622338113863538, 0.8622338113863538]
Accuracy 0.8600608431116906
Recall 0.8475073313782991
Precision 0.9102362204724409


# метод ближайших соседей (для нормализованных данных)

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state = 42)

scores = []
for i in range(1,15):

  knn = KNeighborsClassifier(i)
  knn.fit(X_train, y_train)
  answers = knn.predict(X_test)

  scores.append(knn.score(X_test, y_test))

print(scores)


max_score = max(scores)
scores_ind = [i for i, v in enumerate(scores) if v == max_score]
scores_ind


knn = KNeighborsClassifier(1)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

confusion_matrix(y_test, y_pred)




print("Accuracy" , accuracy_score(y_test, y_pred))
print("Recall" ,recall_score(y_test, y_pred))
print("Precision" , precision_score(y_test, y_pred))

[0.8478922207735767, 0.853541938287701, 0.8609300304215558, 0.8604954367666232, 0.8596262494567579, 0.8613646240764885, 0.8652759669708823, 0.8617992177314211, 0.864406779661017, 0.8617992177314211, 0.8596262494567579, 0.8617992177314211, 0.8657105606258149, 0.8661451542807475]
Accuracy 0.8478922207735767
Recall 0.8504398826979472
Precision 0.888208269525268


# логистическая регрессия с помощью кросс-валидации (для нормализованных данных)

In [ ]:


X_train, X_test, y_train, y_test = train_test_split(X_norm,y, test_size=0.3, random_state = 42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

cvs = cross_val_score(estimator = model, X = X_norm, y = y, cv = kf, scoring = 'accuracy')

print(cvs)
print(cvs.mean())

Accuracy 0.8900478053020426
Recall 0.8585043988269795
Precision 0.9512591389114541


# логистическая регрессия с помощью кросс-валидации (для стандартизированных данных)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_st,y, test_size=0.3, random_state = 42)
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

cvs = cross_val_score(estimator = model, X = X_norm, y = y, cv = kf, scoring = 'accuracy')

print(cvs)
print(cvs.mean())

Accuracy 0.8904823989569752
Recall 0.8599706744868035
Precision 0.9505672609400324


# метод ближайших соседей с помощью кросс-валидации (для нормализованных данных)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state = 42)

scores = []
for i in range(1,15):

  knn = KNeighborsClassifier(i)
  knn.fit(X_train, y_train)
  answers = knn.predict(X_test)

  scores.append(knn.score(X_test, y_test))

print(scores)


max_score = max(scores)
scores_ind = [i for i, v in enumerate(scores) if v == max_score]
scores_ind




knn = KNeighborsClassifier(7)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

confusion_matrix(y_test, y_pred)

print("Accuracy" , accuracy_score(y_test, y_pred))
print("Recall" ,recall_score(y_test, y_pred))
print("Precision" , precision_score(y_test, y_pred)) 


cvs = cross_val_score(estimator = model, X = X_norm, y = y, cv = kf, scoring = 'accuracy')

print(cvs)
print(cvs.mean())

[0.8478922207735767, 0.853541938287701, 0.8609300304215558, 0.8604954367666232, 0.8596262494567579, 0.8613646240764885, 0.8652759669708823, 0.8617992177314211, 0.864406779661017, 0.8617992177314211, 0.8596262494567579, 0.8617992177314211, 0.8657105606258149, 0.8661451542807475]
Accuracy 0.8652759669708823
Recall 0.8409090909090909
Precision 0.925
[0.8559322  0.8559322  0.87092568 0.86049544 0.87345075]
0.8633472556388739


# метод ближайших соседей с помощью кросс-валидации (для стандартизированных данных)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_st, y, test_size=0.3, random_state = 42)

scores = []
for i in range(1,15):

  knn = KNeighborsClassifier(i)
  knn.fit(X_train, y_train)
  answers = knn.predict(X_test)

  scores.append(knn.score(X_test, y_test))

print(scores)


max_score = max(scores)
scores_ind = [i for i, v in enumerate(scores) if v == max_score]
scores_ind




knn = KNeighborsClassifier(7)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

confusion_matrix(y_test, y_pred)




print("Accuracy" , accuracy_score(y_test, y_pred))
print("Recall" ,recall_score(y_test, y_pred))
print("Precision" , precision_score(y_test, y_pred)) 

cvs = cross_val_score(estimator = model, X = X_norm, y = y, cv = kf, scoring = 'accuracy')

print(cvs)
print(cvs.mean())

[0.8383311603650587, 0.8426770969143851, 0.8600608431116906, 0.8504997827031725, 0.8639721860060843, 0.85832246849196, 0.8670143415906127, 0.8626684050412864, 0.8622338113863538, 0.8565840938722294, 0.8600608431116906, 0.8609300304215558, 0.8622338113863538, 0.8622338113863538]
Accuracy 0.8670143415906127
Recall 0.844574780058651
Precision 0.9245585874799358
[0.85397653 0.86701434 0.87157757 0.85853977 0.86823222]
0.8638680876433373


#метод опорных векторов для нормализованных данных

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X_norm, y, test_size=1/3, random_state=42)

svm = SVC()

svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

print("Accuracy:", accuracy_score(y_test,y_pred_svm))
print("Recall:", recall_score(y_test,y_pred_svm))
print("Precision:", precision_score(y_test,y_pred_svm))

Accuracy: 0.8936253421978881
Recall: 0.8536423841059603
Precision: 0.9619402985074627


#метод опорных векторов для стандартизированных данных

In [ ]:


X_train,X_test,y_train,y_test = train_test_split(X_st, y, test_size=1/3, random_state=42)

svm = SVC()

svm.fit(X_train, y_train)
y_pred_svm = svm.predict(X_test)

print("Accuracy:", accuracy_score(y_test,y_pred_svm))
print("Recall:", recall_score(y_test,y_pred_svm))
print("Precision:", precision_score(y_test,y_pred_svm))

Accuracy: 0.8932342588971451
Recall: 0.8529801324503311
Precision: 0.9619118745332338


#Вывод

С помощью данных представленных в датафрейме мы смогли довольно эффективно (с точностью 80-90%)определять в браке человек или нет, следовательно в данном наборе данных существуют довольно сильные, хоть на первый взгляд и не заметные, связи.